In [3]:
#!/usr/bin/env python       

class GetData:
    
    def __init__(self):
        """
        Retrieves and stores the urllib.urlopen object for a given url
        """
        
    def generate_url(self,year):
        
        #generate the url for fetching the log files for every month's first day
        number_of_months=1
        while number_of_months < 13:
            if(number_of_months <10):
                url="http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"+year+"/Qtr1/log"+year+'%02d' % number_of_months+"01.zip"
            else:
                url="http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"+year+"/Qtr1/log"+year+str(number_of_months)+"01.zip"
            number_of_months=number_of_months+1
        #temp_url=download_data("http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr1/log20160101.zip")
        return self.download_data("http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr1/log20160101.zip")
        
    def download_data(self,url):

        import urllib.request
        import requests
        import os.path
        import zipfile
        import pandas as pd

        #fetching the zip file name from the URL
        file_name=url.split("/")

        #Downloading data if not already present in the cache
        if(os.path.exists("Part_2_log_datasets/"+file_name[8])):
            print("Already present")

        else:
            urllib.request.urlretrieve(url, "Part_2_log_datasets/"+file_name[8])
            print("Download complete")

        #unzip the file and fetch the csv file
        zf = zipfile.ZipFile("Part_2_log_datasets/"+file_name[8]) 
        csv_file_name=file_name[8].replace("zip", "csv")
        zf_file=zf.open(csv_file_name)

        #create a dataframe from the csv
        df = pd.read_csv(zf_file)
        return df
        
#fetch the year for which the user wants logs
year = input('Enter the year for which you need to fetch the log files: ')
#calling the function to generate dynamic URL
get_data_obj=GetData()
df=get_data_obj.generate_url(year)
        


Enter the year for which you need to fetch the log files: 2015
Already present


C:\Users\Vasanti\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
print(df.head(25))

                 ip        date      time  zone        cik  \
0     100.43.81.bbb  2016-01-01  00:00:00   0.0   857737.0   
1     100.43.85.jbj  2016-01-01  00:00:00   0.0   910322.0   
2     101.81.76.eib  2016-01-01  00:00:00   0.0   744822.0   
3     101.81.76.eib  2016-01-01  00:00:00   0.0  1403802.0   
4     101.81.76.eib  2016-01-01  00:00:00   0.0  1199070.0   
5     101.81.76.eib  2016-01-01  00:00:00   0.0  1595552.0   
6   107.178.194.aah  2016-01-01  00:00:00   0.0  1204560.0   
7   107.178.194.aah  2016-01-01  00:00:00   0.0  1204560.0   
8   107.178.194.fih  2016-01-01  00:00:00   0.0  1386858.0   
9   107.178.194.fih  2016-01-01  00:00:00   0.0  1386858.0   
10  107.178.194.gfa  2016-01-01  00:00:00   0.0   923120.0   
11  107.178.194.gfa  2016-01-01  00:00:00   0.0   923120.0   
12  115.111.104.igg  2016-01-01  00:00:00   0.0    20740.0   
13   123.125.64.abf  2016-01-01  00:00:00   0.0   768899.0   
14   123.125.64.abf  2016-01-01  00:00:00   0.0   768899.0   
15  130.

In [14]:
import numpy as np

#replacing empty strings with NaN 
df.replace(r'\s+', np.nan, regex=True)

,ip,date,time,zone,cik,accession,extention,code,size,idx,norefer,noagent,find,crawler,browser
0,100.43.81.bbb,2016-01-01,00:00:00,0.0,857737.0,0001564590-15-010715,icon-20150930_cal.xml,200.0,5194.0,0.0,0.0,0.0,10.0,0.0,NaN
1,100.43.85.jbj,2016-01-01,00:00:00,0.0,910322.0,0000899140-15-000782,b15605421.htm,200.0,11840.0,0.0,0.0,0.0,10.0,0.0,NaN
2,101.81.76.eib,2016-01-01,00:00:00,0.0,744822.0,0000878467-15-000834,-index.htm,200.0,67184.0,1.0,0.0,0.0,10.0,0.0,NaN
3,101.81.76.eib,2016-01-01,00:00:00,0.0,1403802.0,0001393905-15-000679,-index.htm,200.0,7019.0,1.0,0.0,0.0,10.0,0.0,NaN
4,101.81.76.eib,2016-01-01,00:00:00,0.0,1199070.0,0001127602-15-035138,-index.htm,200.0,6743.0,1.0,0.0,0.0,10.0,0.0,NaN
5,101.81.76.eib,2016-01-01,00:00:00,0.0,1595552.0,0001477932-15-007840,-index.htm,200.0,12483.0,1.0,0.0,0.0,10.0,0.0,NaN
6,107.178.194.aah,2016-01-01,00:00:00,0.0,1204560.0,0001204560-13-000148,-index.htm,503.0,0.0,1.0,0.0,0.0,10.0,0.0,NaN
7,107.178.194.aah,2016-01-01,00:00:00,0.0,1204560.0,0001204560-13-000148,-index.htm,503.0,4300.0,1.0,0.0,0.0,10.0,0.0,NaN
8,107.178.194.fih,2016-01-01,00:00:00,0.0,1386858.0,0001564590-13-000381,-index.htm,503.0,4254.0,1.0,0.0,0.0,10.0,0.0,NaN
9,107.178.194.fih,2016-01-01,00:00:00,0.0,1386858.0,0001564590-13-000381,-index.htm,503.0,0.0,1.0,0.0,0.0,10.0,0.0,NaN


In [16]:

#replace all ip column NaN value by a default ip address 
df["ip"].fillna("255.255.255.255", inplace=True)

#replace all broser column NaN values by a string
df["browser"].fillna("Not Applicable", inplace=True)


#perform forward fill to replace NaN values by fetching the next valid value
df["date"].fillna(method='ffill')

#perform backward fill to replace NaN values by backpropagating and fetching the previous valid value
df["time"].fillna(method='bfill')

[ 0.]


In [17]:
df['cik'] = df['cik'].astype('int')

In [26]:
df.insert(6, "CIK_Accession_Anamoly_Flag", "N")

In [ ]:
import pandas as pd
import numpy as np

count=0;
for i in df['accession']:
   
    if(df['cik'][count]==i.split("-")[0].lstrip("0")):
        print("its a match")
    count=count+1
    
"""for _, value in df['accession'].iteritems():
    if(df['cik']==value.split("-")[0].lstrip("0")):
        print("Its a match")
if(np.where(df['accession'].values.split("-")[0].lstrip("0")).astype('int')==df['cik']):
    print("its a match")"""
   
    
